In [4]:
codenames_words = ["africa", "agent", "air", "alien", "alps", "amazon", "ambulance", "america", "angel", "antarctica", "apple", "arm", "atlantis", "australia", "aztec", "back", "ball", "band", "bank", "bar", "bark", "bat", "battery", "beach", "bear", "beat", "bed", "beijing", "bell", "belt", "berlin", "bermuda", "berry", "bill", "block", "board", "bolt", "bomb", "bond", "boom", "boot", "bottle", "bow", "box", "bridge", "brush", "buck", "buffalo", "bug", "bugle", "button", "calf", "canada", "cap", "capital", "car", "card", "carrot", "casino", "cast", "cat", "cell", "centaur", "center", "chair", "change", "charge", "check", "chest", "chick", "china", "chocolate", "church", "circle", "cliff", "cloak", "club", "code", "cold", "comic", "compound", "concert", "conductor", "contract", "cook", "copper", "cotton", "court", "cover", "crane", "crash", "cricket", "cross", "crown", "cycle", "czech", "dance", "date", "day", "death", "deck", "degree", "diamond", "dice", "dinosaur", "disease", "doctor", "dog", "draft", "dragon", "dress", "drill", "drop", "duck", "dwarf", "eagle", "egypt", "embassy", "engine", "england", "europe", "eye", "face", "fair", "fall", "fan", "fence", "field", "fighter", "figure", "file", "film", "fire", "fish", "flute", "fly", "foot", "force", "forest", "fork", "france", "game", "gas", "genius", "germany", "ghost", "giant", "glass", "glove", "gold", "grace", "grass", "greece", "green", "ground", "ham", "hand", "hawk", "head", "heart", "helicopter", "himalayas", "hole", "hollywood", "honey", "hood", "hook", "horn", "horse", "horseshoe", "hospital", "hotel", "ice", "icecream", "india", "iron", "ivory", "jack", "jam", "jet", "jupiter", "kangaroo", "ketchup", "key", "kid", "king", "kiwi", "knife", "knight", "lab", "lap", "laser", "lawyer", "lead", "lemon", "leprechaun", "life", "light", "limousine", "line", "link", "lion", "litter", "loch ness", "lock", "log", "london", "luck", "mail", "mammoth", "maple", "marble", "march", "mass", "match", "mercury", "mexico", "microscope", "millionaire", "mine", "mint", "missile", "model", "mole", "moon", "moscow", "mount", "mouse", "mouth", "mug", "nail", "needle", "net", "new york", "night", "ninja", "note", "novel", "nurse", "nut", "octopus", "oil", "olive", "olympus", "opera", "orange", "organ", "palm", "pan", "pants", "paper", "parachute", "park", "part", "pass", "paste", "penguin", "phoenix", "piano", "pie", "pilot", "pin", "pipe", "pirate", "pistol", "pit", "pitch", "plane", "plastic", "plate", "platypus", "play", "plot", "point", "poison", "pole", "police", "pool", "port", "post", "pound", "press", "princess", "pumpkin", "pupil", "pyramid", "queen", "rabbit", "racket", "ray", "revolution", "ring", "robin", "robot", "rock", "rome", "root", "rose", "roulette", "round", "row", "ruler", "satellite", "saturn", "scale", "school", "scientist", "scorpion", "screen", "scuba diver", "seal", "server", "shadow", "shakespeare", "shark", "ship", "shoe", "shop", "shot", "sink", "skyscraper", "slip", "slug", "smuggler", "snow", "snowman", "sock", "soldier", "soul", "sound", "space", "spell", "spider", "spike", "spine", "spot", "spring", "spy", "square", "stadium", "staff", "star", "state", "stick", "stock", "straw", "stream", "strike", "string", "sub", "suit", "superhero", "swing", "switch", "table", "tablet", "tag", "tail", "tap", "teacher", "telescope", "temple", "theater", "thief", "thumb", "tick", "tie", "time", "tokyo", "tooth", "torch", "tower", "track", "train", "triangle", "trip", "trunk", "tube", "turkey", "undertaker", "unicorn", "vacuum", "van", "vet", "wake", "wall", "war", "washer", "washington", "watch", "water", "wave", "web", "well", "whale", "whip", "wind", "witch", "worm", "yard"];

In [142]:
import itertools
import numpy as np
from sklearn.decomposition import PCA


word = []
vec = []

max_words = 30000

with open("/Users/szymon/Downloads/google-10000-english-usa.txt") as f:
    common_words = f.read().strip().split('\n')
    
needed = set(common_words + codenames_words)
with open("/Users/szymon/Downloads/glove.42B.300d.txt") as f:
    for word_idx, line in enumerate(itertools.islice(f, 100000)):
        w, v = line.strip().split(' ', 1)
        v = np.array(list(map(float, v.split(' '))), dtype=np.float32)
        if not w.isalpha() or not w.isascii() or (word_idx >= max_words and w not in needed):
            continue
        word.append(w)
        vec.append(v)
        
# hacks
vec.append((vec[word.index('new')] + vec[word.index('york')]) / 2)
word.append('new york')
vec.append((vec[word.index('loch')] + vec[word.index('ness')]) / 2)
word.append('loch ness')
vec.append((vec[word.index('scuba')] + vec[word.index('diver')]) / 2)
word.append('scuba diver')
# end hacks

vec = np.stack(vec)
word_to_idx = {w: idx for idx, w in enumerate(word)}
def word_to_vec(w):
    return vec[word_to_idx[w]]

vec /= np.linalg.norm(vec, axis=-1)[:, None]
# vec = PCA(n_components=128).fit_transform(vec)
len(word), vec.shape, f'{vec.size * vec.itemsize // 10**6} MB'

(26660, (26660, 300), '31 MB')

In [4]:
%%time
ANALOGIES = [
    ('soft', 'softer', 'hard'),
    ('airplane', 'air', 'boat'),
    ('woman', 'man', 'queen'),
    ('bedroom', 'bed', 'restroom'),
    ('big', 'small', 'long'),
    ('god', 'angel', 'devil'),
    ('computer', 'keyboard', 'car'),
    ('employee', 'boss', 'soldier'),
    ('int', 'float', 'integer'),
    # unit tests for hacky words:
    ('new york', 'usa', 'paris'),
    ('loch ness', 'uk', 'salem'),
    ('scuba diver', 'water', 'astronaut'),
]

def derived_from(word, sources):
    for src in sources:
        if word == src or word == src + 's':
            return True
    return False

for srcfrom, srcto, dstfrom in ANALOGIES:
    targetvec = word_to_vec(srcto) - word_to_vec(srcfrom) + word_to_vec(dstfrom) 
    dist = (1 - (vec * targetvec[None, :])).sum(-1)
    candidates = [word[idx] for idx in dist.argsort()[:100]]
    candidates = [w for w in candidates if not derived_from(w, [srcfrom, srcto, dstfrom])][:5]

    print(f'{srcfrom} -> {srcto} is like {dstfrom} -> {candidates[0]} [{", ".join(candidates[1:])}]')

soft -> softer is like hard -> harder [tougher, easier, faster, better]
airplane -> air is like boat -> water [sea, ocean, river, pump]
woman -> man is like queen -> king [prince, kings, royal, crown]
bedroom -> bed is like restroom -> toilet [toilets, shower, wheelchair, hygiene]
big -> small is like long -> short [longer, relatively, large, few]
god -> angel is like devil -> demon [witch, kitty, raven, ghost]
computer -> keyboard is like car -> truck [pedal, vehicle, suv, guitar]
employee -> boss is like soldier -> hero [boy, man, warrior, army]
int -> float is like integer -> floating [decimal, arithmetic, numeric, binary]
new york -> usa is like paris -> france [belgium, europe, switzerland, germany]
loch ness -> uk is like salem -> usa [united, hampshire, massachusetts, london]
scuba diver -> water is like astronaut -> nasa [space, earth, milk, moon]
CPU times: user 198 ms, sys: 54.7 ms, total: 253 ms
Wall time: 249 ms


In [5]:
glove_words = set(word)
for w in codenames_words:
    if w not in glove_words:
        print(w)

In [10]:
%%time
def get_distances(targetvec):
    return (1 - (vec * targetvec[None, :]).sum(-1))
EXAMPLES = [
    {
        'good': ["church", "cat", "atlantis"],
        'bad': ["eye", "aztec", "buck", "pin", "hospital"],
        'fail': ["fair"],
    },
    {
        'good': ["iron", "ham", "beijing"],
        'bad': ["witch", "note", "cat", "bear", "ambulance"],
        'fail': ["fall"],
    },
    {
        'good': ["robin", "screen", "server"],
        'bad': ["pole", "plate", "ground", "pupil", "iron"],
        'fail': ["novel"],
    }
]

for ex in EXAMPLES:
    good_distances = np.concatenate([get_distances(word_to_vec(w))[:, None] for w in ex['good']], -1)
    bad_distances = np.concatenate([get_distances(word_to_vec(w))[:, None] for w in ex['bad']], -1)
    fail_distances = np.concatenate([get_distances(word_to_vec(w))[:, None] for w in ex['fail']], -1)
    
    risk = 1
    num_guessed = 3
    bad_score = np.minimum(fail_distances[:, 0], np.sort(bad_distances, -1)[:, risk])
    print(bad_score[:5])
    good_score = -np.sort(good_distances, -1)[:, :num_guessed].max(-1)
#     print(good_score[:5])
    print(np.sort(good_distances, -1)[:3])

    score = bad_score + good_score
    candidates = [word[idx] for idx in (-score).argsort()[:1000]][:10]
    print('max score', score.max())
    print('good:', ex['good'])
    print('bad:', ex['bad'])
    print('fail:', ex['fail'])
    print()
    for c in candidates:
        print(c, score[word_to_idx[c]])
    print()
    print('---')
    print()

[0.47582853 0.5133306  0.54044765 0.5138103  0.5261601 ]
[[0.51287854 0.58915204 0.69899464]
 [0.56543154 0.59968007 0.7987106 ]
 [0.5481349  0.60924804 0.7807704 ]]
max score 0.06720805
good: ['church', 'cat', 'atlantis']
bad: ['eye', 'aztec', 'buck', 'pin', 'hospital']
fail: ['fair']

ark 0.06720805
lazarus 0.0562582
aston 0.05493933
jacuzzi 0.05282885
interlude 0.05101335
skydiving 0.048659146
sauna 0.048546195
soundtracks 0.048202276
reactor 0.048033893
cayman 0.047184587

---

[0.42217493 0.5122403  0.44222897 0.48202395 0.48491848]
[[0.5619298  0.66426396 0.7364072 ]
 [0.60418576 0.6906879  0.69164383]
 [0.62309116 0.6565501  0.7291641 ]]
max score 0.11440015
good: ['iron', 'ham', 'beijing']
bad: ['witch', 'note', 'cat', 'bear', 'ambulance']
fail: ['fall']

wok 0.11440015
tong 0.10378122
nonstick 0.086284816
moroccan 0.057478428
ge 0.053643763
btu 0.048133075
gloucester 0.045795977
porto 0.045105636
isi 0.04210043
dynamo 0.039309084

---

[0.5562968  0.57261956 0.5995827  0.56116

In [1]:
with open("/Users/szymon/Downloads/google-10000-english-no-swears.txt") as f:
    common_words_no_swears = f.read().strip().split('\n')

In [9]:
len(word)

26660

In [10]:
b = vec.flatten()#np.array(list(range(1000)), dtype=np.float32).tobytes()
print(len(b))
with open("vec.bytes", 'wb') as f:
    f.write(b)

7998000


In [144]:
other_word_forms = {w: get_forms(w, threshold=0.5) for w in word}

In [151]:
import json

with open("word.json", 'wt') as f:
    json.dump({
        'word': word,
        'common_words': common_words_no_swears,
        'other_word_forms': other_word_forms,
    }, f)

In [29]:
import nltk

from nltk.stem import WordNetLemmatizer 


lemmatizer = WordNetLemmatizer() 
  
print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora")) 
  
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a")) 


rocks : rock
corpora : corpus
better : good


In [43]:
from nltk.corpus import brown
from nltk.tag import UnigramTagger
tagger = UnigramTagger(brown.tagged_sents(categories='news'))

In [79]:
from nltk.corpus import wordnet

ALL_POS_TAGS = [wordnet.ADJ, wordnet.VERB, wordnet.NOUN, wordnet.ADV, wordnet.ADJ_SAT]

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [71]:
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 


In [132]:
def convert(word, from_pos, to_pos):    
    synsets = wn.synsets(word, pos=from_pos)

    # Word not found
    if not synsets:
        return []

    # Get all lemmas of the word (consider 'a'and 's' equivalent)
    lemmas = []
    for s in synsets:
        for l in s.lemmas():
            if s.name().split('.')[1] == from_pos or from_pos in (wordnet.ADJ, wordnet.ADJ_SAT) and s.name().split('.')[1] in (wordnet.ADJ, wordnet.ADJ_SAT):
                lemmas += [l]

    # Get related forms
    derivationally_related_forms = [(l, l.derivationally_related_forms()) for l in lemmas]

    # filter only the desired pos (consider 'a' and 's' equivalent)
    related_noun_lemmas = []

    for drf in derivationally_related_forms:
        for l in drf[1]:
            if l.synset().name().split('.')[1] == to_pos or to_pos in (wordnet.ADJ, wordnet.ADJ_SAT) and l.synset().name().split('.')[1] in (wordnet.ADJ, wordnet.ADJ_SAT):
                related_noun_lemmas += [l]

    # Extract the words from the lemmas
    words = [l.name() for l in related_noun_lemmas]
    len_words = len(words)

    # Build the result in the form of a list containing tuples (word, probability)
    result = [(w, float(words.count(w)) / len_words) for w in set(words)]
    result.sort(key=lambda w:-w[1])

    # return all the possibilities sorted by probability
    return result

def convert_single(*args, **kwargs):
    t = kwargs.pop('threshold', 0.5)
    res = convert(*args, **kwargs)
    if len(res) > 0 and res[0][1] >= t:
        return res[0][0]
    else:
        return None

In [133]:
convert_single('flight', 'n', 'v')

'fly'

In [134]:
convert_single('fight', 'n', 'a'), convert_single('direct', 'a', 'n'), convert_single('medicine', 'n', 'a')

('competitive', 'directness', 'medical')

In [136]:


def get_forms(word, threshold=0.5):
    tag = tagger.tag([word])[0][1]
    if tag is not None:
        tag = get_wordnet_pos(tag)
    if tag is not None:
        res = lemmatizer.lemmatize(word, pos=tag)
    else:
        res = lemmatizer.lemmatize(word)
    forms = [word, res]
    
    if tag is not None:
        for other_tag in ALL_POS_TAGS:
            if other_tag == tag:
                continue
            other_pos_res = convert_single(res, tag, other_tag, threshold=threshold)
            if other_pos_res is not None:
                forms.append(other_pos_res)
    forms.extend([ps.stem(f) for f in forms])
    return sorted(set(forms).difference([word]))

for word in ['website', 'fought', 'better', 'bongs', 'flies', 'medication', 'medic', 'flight']:
    print(word, get_forms(word, threshold=0.5))

website ['websit']
fought ['fight', 'oppon', 'opponent']
better ['good', 'honor']
bongs ['bong']
flies ['fli', 'fly']
medication ['medic']
medic []
flight ['fli', 'fly']


In [ ]:
for w in common_words_no_swears[1000:1100]:
    print(w)
    print('  ', get_forms(w, threshold=0.5))


In [153]:
other_word_forms['webpage']

['webpag']

In [154]:
other_word_forms['website']

['websit']

In [155]:
new_words = ['access', 'action', 'addition', 'adventure', 'advisor', 'airline', 'airport', 'alien', 'american', 'angel', 'ankle', 'apollo', 'archer', 'arena', 'ares', 'arm', 'arrow', 'assassin', 'astronaut', 'author', 'automaton', 'bacon', 'bag', 'ball', 'ballet', 'band', 'banner', 'bar', 'barrel', 'bartender', 'beam', 'bear', 'bell', 'belt', 'bench', 'bet', 'bigfoot', 'bird', 'bite', 'black', 'blade', 'blog', 'boat', 'bolt', 'bomb', 'bone', 'bow', 'bowling', 'box', 'boy', 'break', 'breath', 'bronze', 'brownie', 'browser', 'brush', 'bull', 'bullet', 'butter', 'cable', 'cafe', 'cage', 'cake', 'camera', 'canon', 'cap', 'capitol', 'care', 'carrier', 'cereal', 'ceremony', 'chain', 'character', 'cheek', 'chicken', 'cinema', 'circus', 'city', 'class', 'clone', 'club', 'cod', 'coffee', 'comic', 'connection', 'corn', 'corner', 'corpse', 'cosmos', 'cotton', 'course', 'cow', 'crab', 'cracker', 'crew', 'cross', 'crown', 'crystal', 'cupcake', 'current', 'customer', 'czech', 'dagger', 'death', 'demon', 'devil', 'dinner', 'director', 'directory', 'dolphin', 'dragon', 'drain', 'drive', 'drop', 'eagle', 'egypt', 'elephant', 'elf', 'episode', 'equator', 'face', 'faith', 'fan', 'fantasy', 'farm', 'feather', 'fiction', 'field', 'fight', 'file', 'fire', 'fist', 'floor', 'fly', 'forearm', 'forehead', 'forest', 'fox', 'freezer', 'french', 'frog', 'funeral', 'fur', 'future', 'game', 'garden', 'gate', 'ghost', 'ginger', 'glass', 'glove', 'goat', 'gold', 'golf', 'government', 'gown', 'grill', 'guitar', 'hall', 'hammer', 'hamster', 'hand', 'handle', 'hell', 'hill', 'hole', 'hood', 'hook', 'hose', 'hospital', 'house', 'hunt', 'hymn', 'ice', 'inch', 'inverse', 'italy', 'japan', 'jar', 'jaw', 'jellyfish', 'jersey', 'jerusalem', 'jug', 'jump', 'kennel', 'key', 'kiss', 'kite', 'kitten', 'knight', 'kolkata', 'lake', 'lamp', 'latvia', 'left', 'lid', 'lincoln', 'lion', 'lip', 'lithuania', 'lock', 'lodge', 'london', 'luck', 'magic', 'mango', 'manhattan', 'market', 'martian', 'mathematics', 'maze', 'mermaid', 'mesh', 'meter', 'mind', 'mirror', 'monopoly', 'month', 'moon', 'moose', 'motorcycle', 'mountain', 'movie', 'musical', 'nepal', 'network', 'news', 'night', 'nose', 'note', 'ocean', 'octopus', 'officer', 'opening', 'order', 'oval', 'oven', 'owl', 'pack', 'panda', 'paris', 'park', 'paw', 'paws', 'peach', 'peanut', 'pearl', 'pen', 'pepper', 'performer', 'pharmacist', 'physician', 'pink', 'pirate', 'pit', 'pittsburgh', 'pizza', 'plane', 'plant', 'play', 'pocket', 'poem', 'poker', 'police', 'poodle', 'prayer', 'present', 'priest', 'prince', 'prometheus', 'puppy', 'purple', 'puzzle', 'raccoon', 'radar', 'radio', 'rainbow', 'raptor', 'rat', 'reef', 'reference', 'reindeer', 'report', 'rice', 'river', 'romance', 'room', 'rose', 'round', 'rune', 'salmon', 'salt', 'sand', 'sandals', 'scandinavia', 'scientist', 'screen', 'screw', 'script', 'scuba', 'sea', 'seat', 'second', 'security', 'seoul', 'serpent', 'set', 'shaman', 'shepherd', 'shoe', 'shrimp', 'silk', 'singer', 'single', 'skull', 'sky', 'smile', 'snail', 'sniper', 'soldier', 'soprano', 'soup', 'spaniel', 'spice', 'spider', 'spiderman', 'spoon', 'spring', 'star', 'step', 'stomach', 'stone', 'store', 'storm', 'story', 'student', 'sugar', 'summit', 'surface', 'surgery', 'swan', 'swing', 'syria', 'system', 'tail', 'tea', 'team', 'technician', 'television', 'temperature', 'terrier', 'third', 'throw', 'ticket', 'tie', 'time', 'tissue', 'toe', 'toilet', 'tournament', 'tower', 'track', 'trained', 'trap', 'truck', 'tube', 'tune', 'turtle', 'twilight', 'underwear', 'unit', 'universe', 'update', 'usa', 'vampire', 'vehicle', 'venus', 'violin', 'voice', 'waiter', 'wales', 'war', 'website', 'wedding', 'week', 'well', 'western', 'wind', 'window', 'wing', 'winter', 'wizard', 'wolf', 'world', 'writer', 'yacht', 'year', 'zurich'];


In [156]:
for w in new_words:
    if w not in word:
        print(w)
        

automaton
